<a href="https://colab.research.google.com/github/AugustoCRX/CasePO/blob/main/Case_Pesquisa_Operacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install -q pulp

In [38]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from pulp import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [39]:
import numpy as np
import pandas as pd
from datetime import date, timedelta

# Define a semente para reprodutibilidade dos resultados
np.random.seed(42)

# --- Parâmetros de Configuração ---
num_rooms = 15
num_professionals = 20
start_date = date(2024, 1, 1)
end_date = date(2024, 3, 31)

min_room_availability = 500
max_room_availability = 600
min_session_duration = 30
max_session_duration = 70
min_appointments_per_room_day = 8
max_possible_sessions_per_room_day = 15

min_prof_sessions_per_day = 8
max_prof_sessions_per_day = 12
max_prof_minutes_per_day = 480

mean_idle_time = 60
std_dev_idle_time = 40

# Gera IDs para salas e profissionais
room_ids = [f'Sala_{i:02d}' for i in range(1, num_rooms + 1)]
professional_ids = [f'Profissional_{i:02d}' for i in range(1, num_professionals + 1)]

# Gera todas as datas no intervalo
dates = []
current_date = start_date
while current_date <= end_date:
    dates.append(current_date)
    current_date += timedelta(days=1)

# --- Gera Tabela 1: Resumo da Utilização da Sala (Parâmetros T) ---
table1_data = []

for current_date in dates:
    day = current_date.day
    month = current_date.month
    year = current_date.year

    for room_id in room_ids:
        room_available_time = np.random.randint(min_room_availability, max_room_availability + 1)
        table1_data.append({
            'ID da Sala': room_id,
            'Tempo Disponível da Sala (min)': room_available_time,
            'Dia': day,
            'Mês': month,
            'Ano': year
        })

df_table1 = pd.DataFrame(table1_data)

# --- Gera Tabela 2: Detalhes das Sessões Individuais (Alocações) ---
table2_data = []
professional_daily_load = {} # Rastreia carga diária de cada profissional

for index, row in df_table1.iterrows():
    room_id = row['ID da Sala']
    room_available_time = row['Tempo Disponível da Sala (min)']
    date_obj = date(row['Ano'], row['Mês'], row['Dia'])

    # Determina o tempo total de sessão desejado (T - TempoOcioso)
    min_total_session_time_required = min_appointments_per_room_day * min_session_duration
    max_allowed_idle_time = room_available_time - min_total_session_time_required

    idle_time_for_day = int(np.random.normal(loc=mean_idle_time, scale=std_dev_idle_time))
    idle_time_for_day = np.clip(idle_time_for_day, 1, max_allowed_idle_time - 1)

    target_total_session_time = room_available_time - idle_time_for_day

    # Determina o número de sessões (x) para esta sala neste dia
    lower_bound_x = max(min_appointments_per_room_day, int(np.ceil(target_total_session_time / max_session_duration)))
    upper_bound_x = min(max_possible_sessions_per_room_day, int(np.floor(target_total_session_time / min_session_duration)))

    if lower_bound_x > upper_bound_x:
        num_sessions = min_appointments_per_room_day
        if num_sessions * min_session_duration > target_total_session_time:
            target_total_session_time = num_sessions * min_session_duration
            idle_time_for_day = room_available_time - target_total_session_time
            if idle_time_for_day <= 0:
                idle_time_for_day = 1
                target_total_session_time = room_available_time - 1
    else:
        num_sessions = np.random.randint(lower_bound_x, upper_bound_x + 1)

    num_sessions = np.clip(num_sessions, min_appointments_per_room_day, max_possible_sessions_per_room_day)

    # Gera durações de sessões individuais internamente (não exportadas)
    session_durations_internal = np.random.randint(min_session_duration, max_session_duration + 1, num_sessions)

    # Ajusta as durações internas para que a soma seja o tempo total desejado
    current_sum_durations = sum(session_durations_internal)
    diff = target_total_session_time - current_sum_durations

    if diff != 0:
        indices = np.arange(num_sessions)
        np.random.shuffle(indices)

        for i in range(abs(diff)):
            idx_to_adjust = indices[i % num_sessions]
            if diff > 0: # Adicionar
                if session_durations_internal[idx_to_adjust] < max_session_duration:
                    session_durations_internal[idx_to_adjust] += 1
                else:
                    for k in range(num_sessions):
                        if session_durations_internal[k] < max_session_duration:
                            session_durations_internal[k] += 1
                            break
            else: # Subtrair
                if session_durations_internal[idx_to_adjust] > min_session_duration:
                    session_durations_internal[idx_to_adjust] -= 1
                else:
                    for k in range(num_sessions):
                        if session_durations_internal[k] > min_session_duration:
                            session_durations_internal[k] -= 1
                            break

    # Atribui profissionais a cada sessão
    daily_prof_status = []
    for prof_id in professional_ids:
        prof_key = (prof_id, date_obj)
        current_sessions = professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['sessions']
        current_minutes = professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['minutes']
        daily_prof_status.append({'id': prof_id, 'sessions': current_sessions, 'minutes': current_minutes})

    daily_prof_status.sort(key=lambda x: (x['sessions'], x['minutes']))

    for d_val_internal in session_durations_internal:
        assigned_prof = None

        # Tenta encontrar profissional dentro dos limites de sessões/minutos
        for prof_status in daily_prof_status:
            prof_id = prof_status['id']
            prof_key = (prof_id, date_obj)
            current_sessions = professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['sessions']
            current_minutes = professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['minutes']

            if (current_sessions + 1 <= max_prof_sessions_per_day) and \
               (current_minutes + d_val_internal <= max_prof_minutes_per_day):
                assigned_prof = prof_id
                professional_daily_load[prof_key] = {
                    'sessions': current_sessions + 1,
                    'minutes': current_minutes + d_val_internal
                }
                prof_status['sessions'] += 1
                prof_status['minutes'] += d_val_internal
                break

        # Fallback: Atribui aleatoriamente se os limites estritos não forem atendidos
        if assigned_prof is None:
            assigned_prof = np.random.choice(professional_ids)
            prof_key = (assigned_prof, date_obj)
            professional_daily_load[prof_key] = {
                'sessions': professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['sessions'] + 1,
                'minutes': professional_daily_load.get(prof_key, {'sessions': 0, 'minutes': 0})['minutes'] + d_val_internal
            }
            for prof_status in daily_prof_status:
                if prof_status['id'] == assigned_prof:
                    prof_status['sessions'] += 1
                    prof_status['minutes'] += d_val_internal
                    break

        table2_data.append({
            'ID da Sala': room_id,
            'Dia': date_obj.day,
            'Mês': date_obj.month,
            'Ano': date_obj.year,
            'Professional ID': assigned_prof
        })

df_table2 = pd.DataFrame(table2_data)

In [40]:
df_table1.head()

,ID da Sala,Tempo Disponível da Sala (min),Dia,Mês,Ano
0,Sala_01,551,1,1,2024
1,Sala_02,592,1,1,2024
2,Sala_03,514,1,1,2024
3,Sala_04,571,1,1,2024
4,Sala_05,560,1,1,2024


In [41]:
df_table2.head()

,ID da Sala,Dia,Mês,Ano,Professional ID
0,Sala_01,1,1,2024,Profissional_01
1,Sala_01,1,1,2024,Profissional_01
2,Sala_01,1,1,2024,Profissional_01
3,Sala_01,1,1,2024,Profissional_01
4,Sala_01,1,1,2024,Profissional_01


In [42]:
#Transformar "Professional ID" em OneHotEncoder
df_table2['Professional ID'] = df_table2['Professional ID'].str.split("_").str[1]
df_table2['Professional ID'] = df_table2['Professional ID'].astype(int)

In [43]:
#Transformar as colunas de dia/mês/ano em uma só coluna com datetime64
datetime_format = '%d-%m-%Y'
datetime_series_merged_table1 = pd.to_datetime(df_table1['Dia'].astype(str) + '-' + df_table1['Mês'].astype(str) + '-' + df_table1['Ano'].astype(str), format=datetime_format)
datetime_series_merged_table2 = pd.to_datetime(df_table2['Dia'].astype(str) + '-' + df_table2['Mês'].astype(str) + '-' + df_table2['Ano'].astype(str), format=datetime_format)
df_table1.index = datetime_series_merged_table1
df_table2.index = datetime_series_merged_table2

In [44]:
#Exclui as colunas de dia/mês/ano
df_table1.drop(columns=['Dia', 'Mês', 'Ano'], inplace=True)
df_table2.drop(columns=['Dia', 'Mês', 'Ano'], inplace=True)

In [45]:
rooms_unique = df_table2['ID da Sala'].unique()

In [46]:
#Cria uma tabela auxiliar para calcular a contagem de quantos atendimentos aconteceram na sala X no dia Y
df_table2_room_count = df_table2.reset_index().groupby(['ID da Sala', 'index']).count().reset_index()
df_table2_room_count.index = df_table2_room_count['index']
df_table2_room_count.drop(columns=['index'], inplace=True)
df_table2_room_count.rename(columns={'Professional ID': 'count'}, inplace=True)

In [47]:
# Dicionário das salas

rooms = {}
for room in rooms_unique:
  rooms[room] = {
        date: {'available_time': table1_time, 'times_used': used_times} for date, table1_time, used_times in zip(
            ### Usa-se iloc pois np.where retorna os indices onde o ID da Sala é igual ao room, do loop.
            df_table1.iloc[np.where(df_table1['ID da Sala'] == room)].index,
            df_table1.iloc[np.where(df_table1['ID da Sala'] == room)]['Tempo Disponível da Sala (min)'],
            df_table2_room_count.iloc[np.where(df_table2_room_count['ID da Sala'] == room)]['count'])
  }

#Construção do problema de PO

In [48]:
#Foi necessário criar uma condição adicional ao do case

min_session_duration = 20
max_session_duration = 80

In [49]:
# Vamos seguir o seguinte pipeline:
# Criar um loop sobre a chave de "room" do dicionário
# Criar um loop sobre os dias
# A cada loop de dia, resolver a equação de Programação Linear
# Gerar um resultado com d_med aleatório para comparação
# Armazenar as informações dentro de dicionários separados
# O resultado final será uma média de toda modelagem

results_dict_optimal = {}
results_dict_random = {}

for room in rooms:
    results_dict_optimal[room] = {}
    results_dict_random[room] = {}
    for date in rooms[room].keys():
        # --- Cálculo com a solução ótima (código existente) ---
        # Cria o problema de PL
        prob = LpProblem("IdleRoomProblem", LpMinimize)

        # Define a variável problema da PL
        d_med = LpVariable("DuracaoMedia", lowBound=min_session_duration, upBound=max_session_duration, cat='Integer')

        # Define a equação alvo
        prob += lpSum(rooms[room][date]['available_time'] - d_med * rooms[room][date]['times_used']), "Total Idle Time"

        # Define as restrições
        prob += d_med * rooms[room][date]['times_used'] <= rooms[room][date]['available_time']

        # Executa o solver
        status = prob.solve()
        if LpStatus[prob.status] != 'Optimal':
            print(f'--- Iteração da sala {room} na data {date} não conseguiu obter solução ótima ---')

        # Coleta o resultado ótimo
        obj_optimal = value(prob.objective)

        # Armazena no dicionário de resultados ótimos
        results_dict_optimal[room][date] = obj_optimal

        # --- Cálculo com um valor aleatório para d_med ---
        # Gera um valor aleatório para d_med

        d_med_random = np.random.randint(min_session_duration, max_session_duration + 1) #Condição de tempo máximo e mínimo de sessão

        # Condição que o tempo da sessão tem que ser inferior a multiplicação entre a quantidade de sessões e a duração
        while rooms[room][date]['available_time'] < d_med_random * rooms[room][date]['times_used']:
            d_med_random = np.random.randint(min_session_duration, max_session_duration + 1)
        # Calcula o tempo ocioso com o valor aleatório
        idle_time_random = rooms[room][date]['available_time'] - d_med_random * rooms[room][date]['times_used']

        # Armazena no dicionário de resultados aleatórios
        results_dict_random[room][date] = idle_time_random

In [50]:
results_mean_optimal = {}
results_mean_random = {}

for room in rooms:
    values_optimal = results_dict_optimal[room].values()
    results_mean_optimal[room] = sum(values_optimal) / len(values_optimal)

    values_random = results_dict_random[room].values()
    results_mean_random[room] = sum(values_random) / len(values_random)

    print(f'-----------')
    print(f'Média de tempo ocioso ótima para a sala {room.split("_")[1]}: {results_mean_optimal[room]:.2f}')
    print(f'Média de tempo ocioso aleatória para a sala {room.split("_")[1]}: {results_mean_random[room]:.2f}')
    print(f'Diferença entre as médias: {abs(results_mean_optimal[room] - results_mean_random[room]):.2f}')
    print(f'-----------\n')

-----------
Média de tempo ocioso ótima para a sala 01: 6.18
Média de tempo ocioso aleatória para a sala 01: 152.89
Diferença entre as médias: 146.71
-----------

-----------
Média de tempo ocioso ótima para a sala 02: 4.63
Média de tempo ocioso aleatória para a sala 02: 185.52
Diferença entre as médias: 180.89
-----------

-----------
Média de tempo ocioso ótima para a sala 03: 5.89
Média de tempo ocioso aleatória para a sala 03: 184.11
Diferença entre as médias: 178.22
-----------

-----------
Média de tempo ocioso ótima para a sala 04: 5.60
Média de tempo ocioso aleatória para a sala 04: 135.76
Diferença entre as médias: 130.15
-----------

-----------
Média de tempo ocioso ótima para a sala 05: 5.29
Média de tempo ocioso aleatória para a sala 05: 156.95
Diferença entre as médias: 151.66
-----------

-----------
Média de tempo ocioso ótima para a sala 06: 4.99
Média de tempo ocioso aleatória para a sala 06: 156.41
Diferença entre as médias: 151.42
-----------

-----------
Média de t

In [51]:
fig = make_subplots(rows=5, cols=3, subplot_titles=[f'Tempo ocioso na Sala {sala.split("_")[1]}' for sala in results_dict_optimal])

row = 1
col = 1
# Variáveis para controlar se a legenda já foi mostrada
show_optimal_legend = True
show_random_legend = True

for room in rooms:
    fig.add_trace(go.Scatter(
        x=pd.Series(results_dict_optimal[room].keys()),
        y=pd.Series(results_dict_optimal[room].values()),
        mode='lines',
        name='Ótimo', # Nome genérico para todos os traces ótimos
        line=dict(color='blue'),
        showlegend=show_optimal_legend # Mostra a legenda apenas na primeira vez
    ), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=pd.Series(results_dict_random[room].keys()),
        y=pd.Series(results_dict_random[room].values()),
        mode='lines',
        name='Aleatório', # Nome genérico para todos os traces aleatórios
        line=dict(color='red'),
        showlegend=show_random_legend # Mostra a legenda apenas na primeira vez
    ), row=row, col=col)

    # Após adicionar o primeiro trace de cada tipo, desativa a exibição da legenda
    show_optimal_legend = False
    show_random_legend = False

    row += 1
    if row > 5:
        row = 1
        col += 1
    # A condição de parada pode ser simplificada para parar quando todas as salas forem processadas
    if col > 3 or room == list(rooms.keys())[-1] and row == 1: # Parar após a última sala no último subplot
        break

fig.update_layout(height=1500, width=1500, title_text="Tendência de tempo ocioso por sala")

fig.show()